In [9]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.font_manager as fm
from matplotlib.cm import ScalarMappable

from matplotlib import colors
from descartes import PolygonPatch
from matplotlib.patches import PathPatch
from matplotlib.path import Path

# 設定中文字體
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Microsoft YaHei', 'STHeiti']
plt.rcParams['axes.unicode_minus'] = False

In [10]:
from shapely.geometry import shape, Polygon, MultiPolygon
from shapely import wkt
import math
import aqi_helper as helper

In [11]:
def calc_idw_value(unknowncell, knowncells):
    sumwei = 0.0
    sumpm = 0.0
    wei = 0.0
    for index, row in knowncells.iterrows():
        px = unknowncell['x']
        py = unknowncell['y']
        pm = row['value']
        diss = math.sqrt(math.pow((px - row['lng']), 2) + math.pow((py - row['lat']), 2))
        dis = diss * 345
        if dis < 300:
            if dis < 1:
                dis = 1
            wei = 1 / math.pow(dis, 3)
            sumpm += pm * wei
            sumwei += wei
    if sumwei > 0:
        return round(sumpm / sumwei)
    else:
        return -1

In [12]:
def set_color(value):
    bounds = [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
              28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
              54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
              80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 1000]
    cmap = ['white', '#0000FF', '#001AFF', '#0033FF', '#004DFF', '#0066FF', '#0080FF', '#0099FD', '#00B3FB', '#00CCF9',
            '#00E6F7', '#00FFF5', '#20FFD7', '#40FFB9', '#60FF9B', '#80FF7D', '#9FFF5F', '#BFFF41', '#DFFF23',
            '#FFFF00', '#FFFF00', '#FFFF00', '#FFF800', '#FFF000', '#FFE900', '#FFE100', '#FFDA00', '#FFD200',
            '#FFCB00', '#FFC300', '#FFBC00', '#FFB400', '#FFAD00', '#FFA500', '#FF9E00', '#FF9600', '#FF8F00',
            '#FF8700', '#FF8000', '#FF7800', '#FF7100', '#FF6900', '#FF6200', '#FF5A00', '#FF5300', '#FF4B00',
            '#FF4400', '#FF3C00', '#FF3500', '#FF2D00', '#FF2600', '#FF1E00', '#FF1700', '#FF0F00', '#FF0800',
            '#FF0000', '#FF0000', '#F80000', '#F10000', '#EB0000', '#E40000', '#DD0000', '#D60000', '#CF0000',
            '#C90000', '#C20000', '#BB0000', '#B40000', '#AD0000', '#A70000', '#A00000', '#990000', '#9C000F',
            '#A0001F', '#A3002E', '#A7003D', '#AA004D', '#AD005C', '#B1006B', '#B4007A', '#B8008A', '#BB0099',
            '#BE009E', '#C200A3', '#C500A8', '#C900AD', '#CC00B2', '#CF00B8', '#D300BD', '#D600C2', '#DA00C7',
            '#DD00CC', '#E000D1', '#E400D6', '#E700DB', '#EB00E0', '#EE00E5', '#F100EB', '#F500F0', '#F800F5',
            '#FC00FA', '#FF00FF']

    index = len([i for i in bounds if i > value])
    return cmap[-index]

In [13]:
def calculate_idw(data): #計算網格IDW數值
    df = data #將測站site、lng、lat、value，DataFrame輸入
    df['value'].astype(float)#將value欄位轉成數值型態
    
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.lng, df.lat))
    gdf.crs = 'EPSG:3826'
    gdf['geometry'] = gdf.geometry.buffer(0.015)

    iInterval = (122.7 - 119) / 120
    jInterval = (25.5 - 21.8) / 120
    result = []
    index1 = 0
    index2 = 0
    for j in np.arange(25.5, 21.8, -jInterval):
        nested = []
        for i in np.arange(119, 122.7, iInterval):
            point = {'x': i, 'y': j}
            if helper.is_in_area(index1, index2):
                nested.append(calc_idw_value(point, df))
            else:
                nested.append(-1)
            index2 += 1
        result.append(nested)
        index1 += 1
        index2 = 0

    result = np.array(result)

    return result

In [14]:
def polygon_to_path(polygon):
    """將 Shapely Polygon 轉為 matplotlib 的 Path"""
    vertices = []
    codes = []

    # 外環
    x, y = polygon.exterior.coords.xy
    points = list(zip(x, y))
    vertices.extend(points)
    codes += [Path.MOVETO] + [Path.LINETO] * (len(points) - 2) + [Path.CLOSEPOLY]

    # 內洞（如果有）
    for interior in polygon.interiors:
        x, y = interior.coords.xy
        points = list(zip(x, y))
        vertices.extend(points)
        codes += [Path.MOVETO] + [Path.LINETO] * (len(points) - 2) + [Path.CLOSEPOLY]

    return Path(vertices, codes)

In [15]:
def draw_heatmap(taiwan_map, site2, mean_grid, year):
    iInterval = (122.7 - 119) / 120
    jInterval = (25.5 - 21.8) / 120

    fig, ax = plt.subplots(1)
    ax = taiwan_map.plot(ax=ax, color='none')
    plt.axis('off')
    ax.set_facecolor("none")

    # create discrete colormap
    cmap = colors.ListedColormap(
        ['white', '#0000FF', '#001AFF', '#0033FF', '#004DFF', '#0066FF', '#0080FF', '#0099FD', '#00B3FB', '#00CCF9',
         '#00E6F7', '#00FFF5', '#20FFD7', '#40FFB9', '#60FF9B', '#80FF7D', '#9FFF5F', '#BFFF41', '#DFFF23', '#FFFF00',
         '#FFFF00', '#FFFF00', '#FFF800', '#FFF000', '#FFE900', '#FFE100', '#FFDA00', '#FFD200', '#FFCB00', '#FFC300',
         '#FFBC00', '#FFB400', '#FFAD00', '#FFA500', '#FF9E00', '#FF9600', '#FF8F00', '#FF8700', '#FF8000', '#FF7800',
         '#FF7100', '#FF6900', '#FF6200', '#FF5A00', '#FF5300', '#FF4B00', '#FF4400', '#FF3C00', '#FF3500', '#FF2D00',
         '#FF2600', '#FF1E00', '#FF1700', '#FF0F00', '#FF0800', '#FF0000', '#FF0000', '#F80000', '#F10000', '#EB0000',
         '#E40000', '#DD0000', '#D60000', '#CF0000', '#C90000', '#C20000', '#BB0000', '#B40000', '#AD0000', '#A70000',
         '#A00000', '#990000', '#9C000F', '#A0001F', '#A3002E', '#A7003D', '#AA004D', '#AD005C', '#B1006B', '#B4007A',
         '#B8008A', '#BB0099', '#BE009E', '#C200A3', '#C500A8', '#C900AD', '#CC00B2', '#CF00B8', '#D300BD', '#D600C2',
         '#DA00C7', '#DD00CC', '#E000D1', '#E400D6', '#E700DB', '#EB00E0', '#EE00E5', '#F100EB', '#F500F0', '#F800F5',
         '#FC00FA', '#FF00FF'])
    # bounds = [-1, -0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
    #           27, 28,
    #           29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
    #           55,
    #           56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
    #           82,
    #           83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 1000]
    bounds = [-1, -0.1] + list(range(1, 71)) + [1000]  # 上限為 70
    norm = colors.BoundaryNorm(bounds, cmap.N)

    x1, x2, y1, y2 = 119 - iInterval * 0.5, 122.7 - iInterval * 0.5, 21.8 + jInterval * 0.5, 25.5 + jInterval * 0.5

    print('以行政區邊界裁切熱區圖')
    # 疊加各個區塊並裁切 imshow
    for geom_shape in taiwan_map.geometry:
        if geom_shape.geom_type == 'Polygon':
            path = polygon_to_path(geom_shape)
            patch = PathPatch(path, facecolor='none', edgecolor='#323333', linewidth=0.6)
            ax.add_patch(patch)
            ax.imshow(mean_grid, cmap=cmap, norm=norm,
                    extent=(x1, x2, y1, y2), clip_path=patch, clip_on=True)

        elif geom_shape.geom_type == 'MultiPolygon':
            for subpoly in geom_shape.geoms:
                path = polygon_to_path(subpoly)
                patch = PathPatch(path, facecolor='none', edgecolor='#323333', linewidth=0.6)
                ax.add_patch(patch)
                ax.imshow(mean_grid, cmap=cmap, norm=norm,
                        extent=(x1, x2, y1, y2), clip_path=patch, clip_on=True)
    print('裁切完成')

    print('繪製測站點')
    # 測站點
    df = site2 #將測站site、lng、lat、value，DataFrame輸入
    df['value'].astype(float)#將value欄位轉成數值型態
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.lng, df.lat))
    gdf.crs = 'EPSG:3826'
    gdf['geometry'] = gdf.geometry.buffer(0.015)
    
    ax4 = gdf.plot(ax=ax, edgecolor='#666666', facecolor='none', linewidth=1.5)
    ax4.set_xlim(119.2, 122.08)
    ax4.set_ylim(21.85, 25.4)
    ax4.get_xaxis().set_visible(False)
    ax4.get_yaxis().set_visible(False)

    # 顏色條設定
    sm = ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])  # 避免 colorbar 警告

    # 加入 colorbar（右邊的直條圖例）
    cbar = fig.colorbar(sm, ax=ax, orientation='vertical', shrink=0.65, pad=0.02)
    cbar.set_label('PM2.5 濃度 (μg/m³)', fontsize=12)
    cbar.set_ticks([0, 10, 20, 30, 40, 50, 60, 70])

    fig.set_size_inches(1059.84 / 72, 1186.56 / 72)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0, 0)

    print('測站點繪製完成')
    fig.savefig(f'./month_expand/PM25_heatmap_{year}.png', transparent=True, dpi=72, pad_inches=0)
    plt.close(fig)


In [16]:
# 載入台灣地圖邊界檔 (GML檔案)
taiwan_map = gpd.read_file(r'./TOWN_MOI_1131028.gml')

site = pd.read_csv('./Preview_Data.csv')
site = site[['sitename','twd97lon','twd97lat']]
site.columns = ['site','lng','lat']

excluded_sites = ['富貴角', '馬祖', '金門', '馬公']
site = site[~site['site'].isin(excluded_sites)]

for year in range(2016, 2020, 1):
    f = open(f'./插值後月統計/PM25_{year}_interpolated.csv',encoding='utf8')
    df = pd.read_csv(f)
    df = df.set_index('日期')

    m2 = []
    for count in range(0,len(df),1):#計算每周的120x120網格
        site1 = site.set_index('site')
        df_test = pd.DataFrame(df.iloc[count])
        site1['value'] = df_test
        site2 = site1.reset_index()#將格式整理成site2即可帶入function

        m1 = calculate_idw(site2)
        m2.append(m1)  # 儲存每一週的網格結果
        print(f'{year} 年 月數 {count+1} / {len(df)} 計算完成')
        #----以上程式只補到120x120網格並未包含外島地區，以下程式是參考上述function所寫出來的，並將網格結合擴展成150x150網格-----#

    # 將 m2 轉換為 numpy array，形狀會是 (週數, 120, 120)
    m2_array = np.array(m2)

    # 四捨五入到小數第二位：
    m2_array = np.round(m2_array, 2)

    # 將每週的網格結果輸出為單獨 CSV 檔案
    for week_index, week_grid in enumerate(m2_array, start=1):
        week_df = pd.DataFrame(week_grid)
        week_df.to_csv(f'./grid_csv_month/{year}_month_{week_index}.csv', index=False)
        print(f'✅ 輸出：./grid_csv_month/{year}_month_{week_index}.csv')


2016 年 月數 1 / 12 計算完成
2016 年 月數 2 / 12 計算完成
2016 年 月數 3 / 12 計算完成
2016 年 月數 4 / 12 計算完成
2016 年 月數 5 / 12 計算完成
2016 年 月數 6 / 12 計算完成
2016 年 月數 7 / 12 計算完成
2016 年 月數 8 / 12 計算完成
2016 年 月數 9 / 12 計算完成
2016 年 月數 10 / 12 計算完成
2016 年 月數 11 / 12 計算完成
2016 年 月數 12 / 12 計算完成
✅ 輸出：./grid_csv_month/2016_month_1.csv
✅ 輸出：./grid_csv_month/2016_month_2.csv
✅ 輸出：./grid_csv_month/2016_month_3.csv
✅ 輸出：./grid_csv_month/2016_month_4.csv
✅ 輸出：./grid_csv_month/2016_month_5.csv
✅ 輸出：./grid_csv_month/2016_month_6.csv
✅ 輸出：./grid_csv_month/2016_month_7.csv
✅ 輸出：./grid_csv_month/2016_month_8.csv
✅ 輸出：./grid_csv_month/2016_month_9.csv
✅ 輸出：./grid_csv_month/2016_month_10.csv
✅ 輸出：./grid_csv_month/2016_month_11.csv
✅ 輸出：./grid_csv_month/2016_month_12.csv
2017 年 月數 1 / 12 計算完成
2017 年 月數 2 / 12 計算完成
2017 年 月數 3 / 12 計算完成
2017 年 月數 4 / 12 計算完成
2017 年 月數 5 / 12 計算完成
2017 年 月數 6 / 12 計算完成
2017 年 月數 7 / 12 計算完成
2017 年 月數 8 / 12 計算完成
2017 年 月數 9 / 12 計算完成
2017 年 月數 10 / 12 計算完成
2017 年 月數 11 / 12 計算完成
2017 年 月數 12 / 12 